In [183]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
import torch 
import torch.nn.functional as F
from torch import nn
import pandas as pd
from auto_LiRPA import BoundedModule, BoundedTensor, PerturbationLpNorm
from model_func import *
from torch.utils.data import DataLoader
from PIL import Image, ImageDraw

In [2]:
cd /Users/ncohmpli/Documents/Professionnel/Technical/Verification/Lirpa for IoU/manip/pipeline

/Users/ncohmpli/Documents/Professionnel/Technical/Verification/Lirpa for IoU/manip/pipeline


In [3]:
class NeuralNetwork_BrightnessContrast(nn.Module):
    '''
    New model that takes as input a brightness or constrat value and apply it to a specific image
    using the linear_perturbation layer
    (aka the weight and biases of the linear_perturbation layer are set to specific values to encode
    brightness or contrast for a specific image)
    '''
    def __init__(self, classif=True):
        super(NeuralNetwork_BrightnessContrast, self).__init__()
        self.classif=classif
        seed = 0
        torch.manual_seed(seed)
        padding = 1
        self.linear_perturbation = nn.Linear(1,90*90)
        self.conv0 = nn.Conv2d(1, 16, 3, padding= padding) # 3x3 filters w/ same padding
        self.pool0 = nn.MaxPool2d(2, stride=2)
        self.conv1 = nn.Conv2d(16, 16, 3, padding= padding) # 3x3 filters w/ same padding
        self.pool1 = nn.MaxPool2d(2, stride=2)
        self.flatten = nn.Flatten()
        self.linear_relu_stack= nn.Linear(7744,256)
        if self.classif:
            self.linear = nn.Linear(256, 10)
        else:
            self.linear_all = nn.Linear(256, 4) 

    
    def forward(self, alpha):
        layer = self.linear_perturbation(alpha)
        layer = layer.view((-1, 1, 90, 90))
        layer = self.conv0(layer)
        layer = F.relu(self.pool0(layer))
        layer = self.conv1(layer)
        layer = F.relu(self.pool1(layer))
        layer = self.flatten(layer)
        layer = self.linear_relu_stack(layer)
        layer = F.relu(layer)
        if self.classif:
            logits = self.linear(layer)
        else:
            logits = self.linear_all(layer)
        return logits

In [4]:
model_test = NeuralNetwork_BrightnessContrast(classif=False)
model_test(torch.Tensor([0.001]))

tensor([[-0.1079, -0.0068, -0.0729, -0.0156]], grad_fn=<AddmmBackward0>)

## Digit model

In [5]:
model_digit = NeuralNetwork_BrightnessContrast(classif=True)

filename = 'toy_model_classif'
model_torch_load  = torch.jit.load(f'{filename}.pt')
digit_config = model_torch_load.state_dict()
digit_config['linear_perturbation.weight']=torch.Tensor(np.zeros((90*90, 1), dtype='float32'))
digit_config['linear_perturbation.bias']=torch.Tensor(np.zeros((90*90,), dtype='float32'))
model_digit.load_state_dict(digit_config)
#model_torch_load.state_dict()

<All keys matched successfully>

## Corner Model

In [6]:
model_corners = NeuralNetwork_BrightnessContrast(classif=False)

filename = 'toy_model_corners'
model_torch_load  = torch.jit.load(f'{filename}.pt')
corner_config = model_torch_load.state_dict()
# we create values for the linear_perturbation parameters that will me set for brightness and contrast afterwards
corner_config['linear_perturbation.weight']=torch.Tensor(np.zeros((90*90, 1), dtype='float32'))
corner_config['linear_perturbation.bias']=torch.Tensor(np.zeros((90*90,), dtype='float32'))
model_corners.load_state_dict(corner_config)
#model_torch_load.state_dict()


<All keys matched successfully>

## Load Data

In [7]:
cd utils

/Users/ncohmpli/Documents/Professionnel/Technical/Verification/Lirpa for IoU/manip/pipeline/utils


In [147]:
train_df = pd.read_csv("train.csv")
dict_df_10 = {}
for label in np.unique(list(train_df["label"])): 
    dict_df_10[label] = train_df[train_df["label"] == label].head(10)

In [148]:
train_df_100 = pd.concat(dict_df_10.values())
trainingData = CustomMnistDataset_OL(train_df_100)
train_dataloader = DataLoader(trainingData, batch_size=1, shuffle=False)
iterator = iter(train_dataloader)
print(len(train_dataloader))

100


In [194]:
data = next(iter(train_dataloader))
X_0, _ = data
data = next(iter(train_dataloader))
X_1, Y_1 = data

## Brightness

brightness(x; alpha)= x+ alpha

thus for linear_perturbation

linear_perturbation.weight = [1..1]

linear_perturbation.weight = x

In [153]:
def set_brightness(model, image):
    # create a pytorch model for a specific flatten image
    # once a model has been set for one images, you can call it for any brightness value
    image_flatten = image.view((90*90,))
    brightness_config = model.state_dict()
    brightness_config['linear_perturbation.bias']=image_flatten
    brightness_config['linear_perturbation.weight']=torch.Tensor(np.ones((90*90,1), dtype='float32'))
    model.load_state_dict(brightness_config)
    

## AutoLirpa on brightness

In [154]:
variations_brightness = np.linspace(0, 0.5,101)


In [169]:
def expe_brightness(model_corners, df=train_df_100[:3]): 

    tensor_init_brightness = torch.tensor([[0.0]])
    
    dico_brightness = {"image_id":[], "alpha_range":[], "brightness_variation":[], "lb":[], "ub":[]}

    for image_id in range(len(df)):
            model_lirpa_corners = BoundedModule(model_corners, tensor_init_brightness)
            data = next(test_iterator)
            X, y = data
            set_brightness(model_corners, X)
        
            for brightness_variations in variations_brightness:
                ptb_brightness = PerturbationLpNorm(norm=np.inf, eps=brightness_variations) #attention espilon c'est X_brightness
                input_lirpa_brightness = BoundedTensor(torch.tensor([0.0]), ptb_brightness)
                
                lb_brightness, ub_brightness = model_lirpa_corners.compute_bounds(x=(input_lirpa_brightness,),
                                                                    IBP=True, method='IBP')
                dico_brightness["image_id"].append(image_id)
                dico_brightness["alpha_range"].append(str([tensor_init_brightness.item()-brightness_variations,
                                                           tensor_init_brightness.item()+brightness_variations])) 
                dico_brightness["brightness_variation"].append(brightness_variations)
                dico_brightness["lb"].append(lb_brightness[0].tolist())
                dico_brightness["ub"].append(ub_brightness[0].tolist())
    df = pd.DataFrame(dico_brightness)
    return(df)

df_brightness = expe_brightness(model_corners)

be careful Noémie is doing shit on auto lirpa [BoundInput(name="/0"), BoundParams(name="/1"), BoundParams(name="/2"), BoundParams(name="/3"), BoundParams(name="/4"), BoundParams(name="/5"), BoundParams(name="/6"), BoundParams(name="/7"), BoundParams(name="/8"), BoundParams(name="/9"), BoundParams(name="/10")] ['/0', '/1', '/2', '/3', '/4', '/5', '/6', '/7', '/8', '/9', '/10']
<BoundedTensor: BoundedTensor([0.]), PerturbationLpNorm(norm=inf, eps=0.0)>
BoundInput(name="/0")
be careful Noémie is doing shit on auto lirpa [BoundInput(name="/0"), BoundParams(name="/1"), BoundParams(name="/2"), BoundParams(name="/3"), BoundParams(name="/4"), BoundParams(name="/5"), BoundParams(name="/6"), BoundParams(name="/7"), BoundParams(name="/8"), BoundParams(name="/9"), BoundParams(name="/10")] ['/0', '/1', '/2', '/3', '/4', '/5', '/6', '/7', '/8', '/9', '/10']
<BoundedTensor: BoundedTensor([0.]), PerturbationLpNorm(norm=inf, eps=0.005)>
BoundInput(name="/0")
be careful Noémie is doing shit on auto lirp

In [170]:
df_brightness

,image_id,alpha_range,brightness_variation,lb,ub
0,0,"[0.0, 0.0]",0.000,"[0.4284088611602783, 25.40534210205078, 28.205...","[0.4284088611602783, 25.40534210205078, 28.205..."
1,0,"[-0.005, 0.005]",0.005,"[-62.98200607299805, -38.91437911987305, -35.8...","[199.1392059326172, 220.9141082763672, 341.085..."
2,0,"[-0.01, 0.01]",0.010,"[-118.63693237304688, -86.1573715209961, -68.6...","[433.20489501953125, 439.1376647949219, 704.69..."
3,0,"[-0.015, 0.015]",0.015,"[-194.59742736816406, -151.1396484375, -113.10...","[748.358642578125, 733.5029296875, 1197.742675..."
4,0,"[-0.02, 0.02]",0.020,"[-278.1163024902344, -222.4739990234375, -161....","[1092.9302978515625, 1055.74072265625, 1737.68..."
...,...,...,...,...,...
298,2,"[-0.48, 0.48]",0.480,"[-7978.1962890625, -6716.00732421875, -4656.68...","[32482.111328125, 30627.611328125, 51168.87890..."
299,2,"[-0.485, 0.485]",0.485,"[-8061.44140625, -6786.07666015625, -4705.2915...","[32820.390625, 30946.650390625, 51702.00390625..."
300,2,"[-0.49, 0.49]",0.490,"[-8144.67578125, -6856.12744140625, -4753.8833...","[33158.6015625, 31265.619140625, 52235.0078125..."
301,2,"[-0.495, 0.495]",0.495,"[-8227.8916015625, -6926.16845703125, -4802.46...","[33496.7265625, 31584.51953125, 52767.90234375..."


In [171]:
df_brightness[df_brightness["image_id"]==0]

,image_id,alpha_range,brightness_variation,lb,ub
0,0,"[0.0, 0.0]",0.000,"[0.4284088611602783, 25.40534210205078, 28.205...","[0.4284088611602783, 25.40534210205078, 28.205..."
1,0,"[-0.005, 0.005]",0.005,"[-62.98200607299805, -38.91437911987305, -35.8...","[199.1392059326172, 220.9141082763672, 341.085..."
2,0,"[-0.01, 0.01]",0.010,"[-118.63693237304688, -86.1573715209961, -68.6...","[433.20489501953125, 439.1376647949219, 704.69..."
3,0,"[-0.015, 0.015]",0.015,"[-194.59742736816406, -151.1396484375, -113.10...","[748.358642578125, 733.5029296875, 1197.742675..."
4,0,"[-0.02, 0.02]",0.020,"[-278.1163024902344, -222.4739990234375, -161....","[1092.9302978515625, 1055.74072265625, 1737.68..."
...,...,...,...,...,...
96,0,"[-0.48, 0.48]",0.480,"[-7977.18359375, -6691.99853515625, -4657.3911...","[32307.69140625, 30516.904296875, 50954.125, 4..."
97,0,"[-0.485, 0.485]",0.485,"[-8060.3720703125, -6761.99072265625, -4705.96...","[32645.482421875, 30835.548828125, 51486.5625,..."
98,0,"[-0.49, 0.49]",0.490,"[-8143.552734375, -6831.97900390625, -4754.527...","[32983.25, 31154.169921875, 52018.95703125, 50..."
99,0,"[-0.495, 0.495]",0.495,"[-8226.724609375, -6901.96240234375, -4803.094...","[33320.9921875, 31472.755859375, 52551.3125, 5..."


In [160]:
print("For each images, this is the number of experiments:")
pd.DataFrame(df_brightness.groupby(["image_id"]).agg("count")["alpha_range"]).rename(columns={"alpha_range":"nb_exp"})

For each images, this is the number of experiments:


,nb_exp
image_id,
0,101
1,101
2,101


In [162]:
X_0.shape

torch.Size([1, 1, 90, 90])